<a href="https://colab.research.google.com/github/AleenaTerasa/Hello/blob/main/chest%20x%20rays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications import MobileNetV2



In [ ]:
model = MobileNetV2(weights='imagenet', include_top=False)


/tmp/ipython-input-3777817333.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
import os
os.listdir()

['.config', 'archive (1).zip', 'sample_data']

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aleenaterasa","key":"d517aaa78f5c56d1b8f6589cbdf67f4e"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets list -s covid

ref                                                   title                                                      size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------  --------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
imdevskp/corona-virus-report                          COVID-19 Dataset                                       19921674  2020-08-07 03:47:47.907000         416770       2777  1.0              
meirnizri/covid19-dataset                             COVID-19 Dataset                                        4890659  2022-11-13 15:47:17.437000          69043        872  1.0              
sudalairajkumar/covid19-in-india                      COVID-19 in India                                        776022  2021-08-11 04:14:10.130000         308876       2101  0.9117647        
tawsifurrahman/covid19-radiography-database  

In [5]:
!kaggle datasets download -d preetviradiya/covid19-radiography-dataset

Dataset URL: https://www.kaggle.com/datasets/preetviradiya/covid19-radiography-dataset
License(s): GPL-2.0
 91% 678M/747M [00:06<00:01, 52.8MB/s]
100% 747M/747M [00:06<00:00, 112MB/s] 


In [6]:
import zipfile

with zipfile.ZipFile('covid19-radiography-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [8]:
import os
os.listdir()

['.config',
 'covid19-radiography-dataset.zip',
 'kaggle.json',
 'metadata.csv',
 'COVID-19_Radiography_Dataset',
 'sample_data']

In [13]:
os.listdir('COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset')

['Lung_Opacity', 'Viral Pneumonia', 'COVID', 'Normal']

In [14]:
from sklearn.model_selection import train_test_split
import shutil
import os

base_dir = 'COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset'
new_base = 'covid_binary'

os.makedirs(new_base+'/train/COVID', exist_ok=True)
os.makedirs(new_base+'/train/NORMAL', exist_ok=True)
os.makedirs(new_base+'/test/COVID', exist_ok=True)
os.makedirs(new_base+'/test/NORMAL', exist_ok=True)

In [15]:
covid_files = os.listdir(base_dir+'/COVID')[:1500]
normal_files = os.listdir(base_dir+'/Normal')[:1500]

In [16]:
covid_train, covid_test = train_test_split(covid_files, test_size=0.2, random_state=42)
normal_train, normal_test = train_test_split(normal_files, test_size=0.2, random_state=42)

In [17]:
for file in covid_train:
    shutil.copy(base_dir+'/COVID/'+file, new_base+'/train/COVID/'+file)

for file in covid_test:
    shutil.copy(base_dir+'/COVID/'+file, new_base+'/test/COVID/'+file)

for file in normal_train:
    shutil.copy(base_dir+'/Normal/'+file, new_base+'/train/NORMAL/'+file)

for file in normal_test:
    shutil.copy(base_dir+'/Normal/'+file, new_base+'/test/NORMAL/'+file)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'covid_binary/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'covid_binary/test',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [19]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [20]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=base_model.input, outputs=output)

In [21]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [22]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=3
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
75/75 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.7807 - loss: 0.4737 - val_accuracy: 0.8900 - val_loss: 0.2521
Epoch 2/3
75/75 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.8958 - loss: 0.2381 - val_accuracy: 0.9183 - val_loss: 0.1917
Epoch 3/3
75/75 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.9307 - loss: 0.1711 - val_accuracy: 0.8967 - val_loss: 0.2188


ADDING GAUSSIAN BLUR


In [23]:
import cv2
import numpy as np

def gaussian_blur(img):
    return cv2.GaussianBlur(img, (5,5), 0)

In [24]:
train_datagen_blur = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=gaussian_blur
)

test_datagen_blur = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=gaussian_blur
)

train_generator_blur = train_datagen_blur.flow_from_directory(
    'covid_binary/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

test_generator_blur = test_datagen_blur.flow_from_directory(
    'covid_binary/test',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


rebuilding model (blurred version)

In [25]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
base_model_blur = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model_blur.trainable = False

add classifier layers

In [27]:
x = base_model_blur.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_blur = models.Model(inputs=base_model_blur.input, outputs=output)

In [28]:
model_blur.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

training blurred model

In [29]:
history_blur = model_blur.fit(
    train_generator_blur,
    validation_data=test_generator_blur,
    epochs=3
)

Epoch 1/3
75/75 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.7325 - loss: 0.5282 - val_accuracy: 0.8833 - val_loss: 0.2954
Epoch 2/3
75/75 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.8831 - loss: 0.2838 - val_accuracy: 0.8583 - val_loss: 0.3644
Epoch 3/3
75/75 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.8999 - loss: 0.2401 - val_accuracy: 0.8817 - val_loss: 0.2835
